# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Inicia o mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh

# Ajusta os landmarks do rosto para imagens
face_mesh_images = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2, min_detection_confidence=0.5)

# Ajusta os landmarks do rosto para videos
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5,min_tracking_confidence=0.5)

# Inicia a classe de desenho do mediapipe
#mp_drawing = mp.solutions.drawing_utils

# Inicia a classe de estilos de desenho do mediapipe
mp_drawing_styles = mp.solutions.drawing_styles

# 1. Make Some Detections

In [3]:
#Seleciona camera
cap = cv2.VideoCapture(0)
# Inicia o holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # Aplica FACEMESH CONTOURS
        mp_drawing.draw_landmarks(image, results.face_landmarks ,mp_holistic.FACEMESH_CONTOURS,
                                    landmark_drawing_spec=None, 
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [4]:
#results.face_landmarks

In [5]:
results.face_landmarks.landmark[0].visibility

0.0

# 2. Capture Landmarks & Export to CSV
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<!--<img src="https://i.imgur.com/AzKNp7A.png">-->

In [6]:
import csv
import os
import numpy as np

In [7]:
#num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords = len(results.face_landmarks.landmark)
num_coords

468

In [8]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [9]:
#landmarks

In [10]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [15]:
class_name = "Sad"

In [16]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # Aplica FACEMESH CONTOURS
        mp_drawing.draw_landmarks(image, results.face_landmarks ,mp_holistic.FACEMESH_CONTOURS,
                                    landmark_drawing_spec=None, 
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        # Export coordinates
        try:
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_csv('coords.csv')

In [19]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z466,v466,x467,y467,z467,v467,x468,y468,z468,v468
0,Happy,0.440451,0.539810,-0.029466,0.0,0.443100,0.496554,-0.073214,0.0,0.441369,...,-0.018714,0.0,0.529503,0.375795,0.009115,0.0,0.536150,0.368960,0.009829,0.0
1,Happy,0.441566,0.542283,-0.028448,0.0,0.444213,0.497294,-0.073013,0.0,0.442270,...,-0.019056,0.0,0.527416,0.377099,0.009305,0.0,0.533810,0.371335,0.010033,0.0
2,Happy,0.441019,0.542987,-0.028195,0.0,0.443762,0.497351,-0.072836,0.0,0.441789,...,-0.019062,0.0,0.526712,0.376929,0.008769,0.0,0.533011,0.371608,0.009435,0.0
3,Happy,0.440799,0.539870,-0.028672,0.0,0.443137,0.494611,-0.072715,0.0,0.441526,...,-0.018862,0.0,0.527190,0.376773,0.008358,0.0,0.533583,0.371224,0.009068,0.0
4,Happy,0.441217,0.540954,-0.028156,0.0,0.443791,0.495901,-0.071884,0.0,0.441997,...,-0.018763,0.0,0.526450,0.377643,0.008395,0.0,0.532818,0.371771,0.009111,0.0


In [20]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z466,v466,x467,y467,z467,v467,x468,y468,z468,v468
315,Sad,0.544104,0.687628,-0.000181,0.0,0.565849,0.641538,-0.050122,0.0,0.547311,...,-0.029795,0.0,0.606426,0.476049,0.014703,0.0,0.610541,0.466661,0.016607,0.0
316,Sad,0.542976,0.687278,-0.000506,0.0,0.564572,0.642402,-0.050004,0.0,0.546156,...,-0.029865,0.0,0.604941,0.477095,0.013835,0.0,0.609279,0.466838,0.015772,0.0
317,Sad,0.541990,0.686421,-0.001494,0.0,0.564116,0.641556,-0.049769,0.0,0.545574,...,-0.028580,0.0,0.603709,0.476997,0.015195,0.0,0.608008,0.467524,0.017080,0.0
318,Sad,0.542585,0.685856,-0.001418,0.0,0.564624,0.641270,-0.049798,0.0,0.546105,...,-0.028724,0.0,0.603687,0.476436,0.014991,0.0,0.607940,0.467068,0.016828,0.0
319,Sad,0.542270,0.683213,-0.001287,0.0,0.564025,0.637987,-0.050891,0.0,0.545710,...,-0.029427,0.0,0.604104,0.472423,0.014298,0.0,0.608210,0.462911,0.016180,0.0


In [21]:
df[df['class']=='Sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z466,v466,x467,y467,z467,v467,x468,y468,z468,v468
159,Sad,0.451722,0.646359,-0.014624,0.0,0.457469,0.598680,-0.063243,0.0,0.453393,...,-0.028710,0.0,0.544315,0.457767,-0.002250,0.0,0.550113,0.450407,-0.002300,0.0
160,Sad,0.451148,0.649854,-0.015102,0.0,0.454650,0.607109,-0.062308,0.0,0.451694,...,-0.027238,0.0,0.541667,0.457569,-0.001950,0.0,0.547550,0.450227,-0.001913,0.0
161,Sad,0.448440,0.651262,-0.012631,0.0,0.451276,0.606864,-0.061116,0.0,0.448836,...,-0.029182,0.0,0.540692,0.457206,-0.005025,0.0,0.546376,0.449954,-0.005116,0.0
162,Sad,0.449517,0.650469,-0.013045,0.0,0.452369,0.606356,-0.061083,0.0,0.449789,...,-0.028485,0.0,0.539860,0.454313,-0.004263,0.0,0.545449,0.447344,-0.004389,0.0
163,Sad,0.448563,0.650349,-0.013591,0.0,0.450830,0.604930,-0.061323,0.0,0.448605,...,-0.028266,0.0,0.539917,0.457053,-0.004141,0.0,0.545566,0.449994,-0.004240,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Sad,0.544104,0.687628,-0.000181,0.0,0.565849,0.641538,-0.050122,0.0,0.547311,...,-0.029795,0.0,0.606426,0.476049,0.014703,0.0,0.610541,0.466661,0.016607,0.0
316,Sad,0.542976,0.687278,-0.000506,0.0,0.564572,0.642402,-0.050004,0.0,0.546156,...,-0.029865,0.0,0.604941,0.477095,0.013835,0.0,0.609279,0.466838,0.015772,0.0
317,Sad,0.541990,0.686421,-0.001494,0.0,0.564116,0.641556,-0.049769,0.0,0.545574,...,-0.028580,0.0,0.603709,0.476997,0.015195,0.0,0.608008,0.467524,0.017080,0.0
318,Sad,0.542585,0.685856,-0.001418,0.0,0.564624,0.641270,-0.049798,0.0,0.546105,...,-0.028724,0.0,0.603687,0.476436,0.014991,0.0,0.607940,0.467068,0.016828,0.0


In [22]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [24]:
y_test

73       Happy
7        Happy
250        Sad
304        Sad
95     Neutral
        ...   
92     Neutral
205        Sad
110    Neutral
5        Happy
167        Sad
Name: class, Length: 96, dtype: object

## 3.2 Train Machine Learning Classification Model

In [25]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [26]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [27]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [28]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [29]:
fit_models['rc'].predict(X_test)

array(['Happy', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Neutral',
       'Happy', 'Sad', 'Sad', 'Sad', 'Neutral', 'Neutral', 'Happy',
       'Happy', 'Neutral', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Sad',
       'Sad', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Sad', 'Neutral',
       'Happy', 'Neutral', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Neutral', 'Happy', 'Sad', 'Sad',
       'Neutral', 'Neutral', 'Happy', 'Neutral', 'Neutral', 'Happy',
       'Neutral', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Happy',
       'Happy', 'Neutral', 'Sad', 'Happy', 'Neutral', 'Neutral',
       'Neutral', 'Happy', 'Neutral', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Neutral', 'Sad', 'Neutral', 'Sad', 'Sad',
       'Neutral', 'Sad', 'Neutral', 'Sad', 'Neutral', 'Happy', 'Sad'],
      dtype='<U7')

## 3.3 Evaluate and Serialize Model 

In [30]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [31]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [32]:
fit_models['rf'].predict(X_test)

array(['Happy', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Neutral',
       'Happy', 'Sad', 'Sad', 'Sad', 'Neutral', 'Neutral', 'Happy',
       'Happy', 'Neutral', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Sad',
       'Sad', 'Happy', 'Sad', 'Sad', 'Neutral', 'Sad', 'Sad', 'Neutral',
       'Happy', 'Neutral', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Neutral', 'Happy', 'Sad', 'Sad',
       'Neutral', 'Neutral', 'Happy', 'Neutral', 'Neutral', 'Happy',
       'Neutral', 'Happy', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy',
       'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad', 'Happy',
       'Happy', 'Neutral', 'Sad', 'Happy', 'Neutral', 'Neutral',
       'Neutral', 'Happy', 'Neutral', 'Sad', 'Sad', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Neutral', 'Sad', 'Neutral', 'Sad', 'Sad',
       'Neutral', 'Sad', 'Neutral', 'Sad', 'Neutral', 'Happy', 'Sad'],
      dtype=object)

In [33]:
y_test

73       Happy
7        Happy
250        Sad
304        Sad
95     Neutral
        ...   
92     Neutral
205        Sad
110    Neutral
5        Happy
167        Sad
Name: class, Length: 96, dtype: object

In [34]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [35]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [36]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # Aplica FACEMESH CONTOURS
        mp_drawing.draw_landmarks(image, results.face_landmarks ,mp_holistic.FACEMESH_CONTOURS,
                                    landmark_drawing_spec=None, 
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                                 
        # Export coordinates
        try:

            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()